## Notes on seminar 5
### Starring: functions
#### Co-stars: generator functions, decorators, scopes

## 1. Basic function syntax

In [1]:
def sum_two(a, b):
    return a + b

In [2]:
sum_two(1, 2)

In [5]:
sum_two('1', '123')

'1123'

In [6]:
def sum_two_optional(a, b=2):
    print(a + b)  # no return statement

In [7]:
print(sum_two_optional(1))

3
None


### 1.1 Function as an object

In [8]:
def hello():
    '''Prints a greeting'''
    print('Hello world')

# Let's make a little dossier
print('Name:', hello.__name__)
print('Docstring:', hello.__doc__)
print('Type:', type(hello))
print('Id:', id(hello))
print('Hash:', hash(hello))

Name: hello
Docstring: Prints a greeting
Type: <class 'function'>
Id: 139836826839376
Hash: 8739801677461


In [9]:
print(hello.__code__)

<code object hello at 0x7f2e4c5c5780, file "<ipython-input-8-e561bfc9e072>", line 1>


In [66]:
from math import sqrt

# Even a standard function can be hacked :)
sqrt.__doc__

'sqrt(x)\n\nReturn the square root of x.'

It's possible to assign and reassign functions:

In [15]:
please_print = print
please_print('Hey you!')

Hey you!


In [18]:
print = (lambda x: None)

In [19]:
print('Print')  # nothing happens there
please_print('Please, print!!!')

Please, print!!!


In [20]:
print = please_print  # restore it back

### 1.2 Variable number or arguments (a.k.a args & kwargs)

In [21]:
def sum_any_number_of_args(a, b=2, *args):
    print('a:', a)
    print('b:', b)
    print('args:', args)
    return a + b + sum(args)

In [22]:
sum_any_number_of_args(1, 2, 3, 4)

a: 1
b: 2
args: (3, 4)


10

In [23]:
sum_any_number_of_args(1, 2)

a: 1
b: 2
args: ()


3

In [24]:
sum_any_number_of_args(1)

a: 1
b: 2
args: ()


3

In [25]:
def sum_any_number_of_args_and_kwargs(a, b=2, *args, **kwargs):
    print('a:', a)
    print('b:', b)
    print('args:', args)
    print('kwargs:', kwargs)
    return a + b + sum(args) + sum(kwargs.values())

In [26]:
sum_any_number_of_args_and_kwargs(1, 2, 3, value1=10)

a: 1
b: 2
args: (3,)
kwargs: {'value1': 10}


16

In [27]:
sum_any_number_of_args_and_kwargs(*list(range(5)))

a: 0
b: 1
args: (2, 3, 4)
kwargs: {}


10

In [28]:
sum_any_number_of_args_and_kwargs(0, 1, **{'one': 1, 'two': 2})

a: 0
b: 1
args: ()
kwargs: {'one': 1, 'two': 2}


4

In [29]:
def add_len(iterable=[]):
    print('Before:', iterable)
    iterable.append(len(iterable))
    print('After:', iterable)

In [30]:
add_len([])
add_len([])

Before: []
After: [0]
Before: []
After: [0]


#### What can happen when your default argument is a mutable object? Actually, a lot

In [31]:
add_len()
add_len()
add_len()

Before: []
After: [0]
Before: [0]
After: [0, 1]
Before: [0, 1]
After: [0, 1, 2]


**How we would fix it production-like style?**

In [32]:
def add_len(iterable=None):
    if iterable is None:
        iterable = []
    if not isinstance(iterable, list):
        raise ValueError('bad value: {}'.format(iterable))
    
    print('Before:', iterable)
    iterable.append(len(iterable))
    print('After:', iterable)

In [33]:
add_len()
add_len()
add_len()

Before: []
After: [0]
Before: []
After: [0]
Before: []
After: [0]


### 1.3 Anonymous function: lambda

In [34]:
def maths(a, b, operation):
    '''
    :param a: numeric value1
    :param b: numeric value2
    :param operation: function
    :return: operation of a and b
    '''
    return operation(a, b)

In [35]:
maths(1, 2, lambda x, y: x + y)  # we could also define a normal function and pass it here

3

## 2. Generator functions
So-called coroutines

In [36]:
def my_little_range(a, b, step=1):
    tmp = a
    while tmp < b:
        print('yielding', tmp)
        yield tmp
        tmp += step

In [39]:
import time

for i in my_little_range(1, 10, 2):
    print(i)
    time.sleep(1)  # just to show how it works

yielding 1
1
yielding 3
3
yielding 5
5
yielding 7
7
yielding 9
9


In [43]:
print(type(my_little_range(1, 10)))
print(type(range(1, 10)))  # range has a special class but it is a generator by nature

<class 'generator'>
<class 'range'>


**When iterator has no more elements to yield, it throws a StopIteration exception**

In [45]:
generator = my_little_range(0, 3)

while True:
    next(generator)

yielding 0
yielding 1
yielding 2


StopIteration: 

## 3. Decorators
Syntactic sugar. Typical usage: checking arguments, time measuring, logging, etc.

Decorator is usually a function, but it can also be defined as an object with `__call__` method.

Nice article series (in Russian): https://habr.com/ru/post/141411/

In [63]:
def decorator(f):
    def decorated_function(*args, **kwargs):
        print('begin')
        answer = f(*args, **kwargs)
        print('end')
        return answer
    
    return decorated_function

In [64]:
@decorator
def hello():
    print('Name:', hello.__name__)  # something bad happened, we've lost a name!
    print('Hello')

hello()

begin
Name: decorated_function
Hello
end


In [65]:
def decorator(f):
    def decorated_function(*args, **kwargs):
        print('begin')
        answer = f(*args, **kwargs)
        print('end')
        return answer
    
    decorated_function.__name__ = f.__name__  # no worries, we can keep it!
    return decorated_function

@decorator
def hello():
    print('Name:', hello.__name__)
    print('Hello')

hello()

begin
Name: hello
Hello
end


**Let's make a decorator preventing function from being called more than once**

**Variant 1:**

In [53]:
def call_once(f):
    called = False
    
    def decorated_function(*args, **kwargs):
        nonlocal called
        if not called:
            called = True
            return f(*args, **kwargs)
        else:
            print('sorry but no')
    
    return decorated_function

**Variant 2:**

In [54]:
def call_once(f):
    def decorated_function(*args, **kwargs):
        if not hasattr(f, 'called'):
            f.called = True
            return f(*args, **kwargs)
        else:
            print('sorry but no')
    
    return decorated_function

In [55]:
@call_once
def hello():
    print('Hello')

In [56]:
hello()
hello()
hello()

Hello
sorry but no
sorry but no


### 3.1 Parametrized decorators
\+example for functools.wraps

\+example for multiple decorators

In [78]:
import functools

def sandwich(layer):
    # Yes, it's a function that returns decorator (that wraps a function)!
    
    def wraps(func):
        @functools.wraps(func)
        def wrapper(*args, **kwargs):
            print(layer)
            result = func(*args, **kwargs)
            print(layer)
            return result
        return wrapper
    
    return wraps

@sandwich('bread')
@sandwich('salad')
@sandwich('cheese') 
def func():
    print('SECRET')    

func()

bread
salad
cheese
SECRET
cheese
salad
bread


## 4. Scope

**At the root level all globals are actually locals**

In [67]:
globals() == locals()

True

**Conditions and cycles don't create new scopes...**

In [70]:
if 2 * 2 == 4:
    print(globals() == locals())

True


**...but functions definitely do!**

In [73]:
a = 100
c = 100

def func(a, b):
    global c
    c = 100500
    a = 100500
    print(locals())

func(1, 2)
print(a)  # didn't change
print(c)  # changed

{'b': 2, 'a': 100500}
100
100500


## At last: a little trick

In [ ]:
def factorial(n):
    if n in (0, 1):
        return 1
    return n * factorial(n - 1)

In [ ]:
factorial(5)

In [ ]:
factorial2 = factorial
factorial2(5)

In [ ]:
id(factorial) == id(factorial2)

In [ ]:
def factorial(n):
    return 1

factorial(5)

In [ ]:
factorial2(5)  # WTF?! Guess what went wrong